In [1]:
import numpy as np
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
from google.colab import files
files.upload()

In [3]:
DATA_FILES="/content/human_ai_data.json"
MODEL_NAME="microsoft/deberta-v3-base"
OUTPUT_DIR = "./ai_detector_model"
BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 2e-5

In [4]:
from datasets import DatasetDict

print(f"Loading data from {DATA_FILES}...")
# Load JSON directly into a Hugging Face Dataset
dataset = load_dataset("json", data_files=DATA_FILES, split="train")

Loading data from /content/human_ai_data.json...


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
print("Total samples:", len(dataset))


Total samples: 3641


In [6]:
# 2. Convert to Pandas to easily drop duplicates
df = dataset.to_pandas()
initial_count = len(df)

# Drop duplicates based on the 'text' column
df = df.drop_duplicates(subset=["text"])
final_count = len(df)

print(f"Removed {initial_count - final_count} duplicate rows.")

# 3. Convert back to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

Removed 0 duplicate rows.


# Train Test split

In [7]:


train_testvalid = dataset.train_test_split(test_size=0.2, seed=42)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)

final_datasets = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']
})

print(f"Data Split Complete:")
print(f"Train size: {len(final_datasets['train'])} (80%)")
print(f"Valid size: {len(final_datasets['validation'])} (10%)")
print(f"Test size:  {len(final_datasets['test'])} (10%)")

Data Split Complete:
Train size: 2912 (80%)
Valid size: 364 (10%)
Test size:  365 (10%)


In [8]:
train_texts = set(final_datasets['train']["text"])
val_texts   = set(final_datasets['validation']["text"])

print(len(train_texts & val_texts))


0


In [9]:
from collections import Counter
Counter(final_datasets['validation']["label"])


Counter({1: 155, 0: 209})

# Tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
)

print("Tokenizing data...")
tokenized_datasets = final_datasets.map(preprocess_function, batched=True)


# Model

In [ ]:
print("Initializing model...")
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="binary"
      )
    acc = accuracy_score(labels, predictions)

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

# Training Arguments

In [13]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,
    logging_steps=50,
    fp16=True,
    report_to="none",
)

# Model Training

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipython-input-2311307804.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.090000,0.292540,0.947802,0.942249,0.890805,1.000000
2,0.010000,0.031103,0.994505,0.993590,0.987261,1.000000
3,0.000500,0.078408,0.989011,0.987261,0.974843,1.000000


TrainOutput(global_step=546, training_loss=0.06967392053127616, metrics={'train_runtime': 818.4612, 'train_samples_per_second': 10.674, 'train_steps_per_second': 0.667, 'total_flos': 2298579401244672.0, 'train_loss': 0.06967392053127616, 'epoch': 3.0})

# Testing Model

In [31]:
import random
import json
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# 1. Load your trained model (Epoch 2 checkpoint)
checkpoint_path = "/content/ai_detector_model/checkpoint-364"  # Update if needed
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# 2. Load one random AI sample from YOUR generated file
with open("human_ai_data.json", "r") as f:
    data = json.load(f)
    # If data is a dict {'data': [...]}, adjust accordingly. Assuming list based on previous steps.
    if isinstance(data, dict): data = data.get('data', [])

    # Filter for AI only (Label 1)
    ai_samples = [d['text'] for d in data if d.get('label') == 1]

# Pick a random one
test_text = random.choice(ai_samples)

print(f"--- TESTING ON TRAINED DISTRIBUTION (GPT-Neo) ---")
print(f"Text Snippet: {test_text[:100]}...")
result = classifier(test_text)
print(f"Result: {result}")

The tokenizer you are loading from '/content/ai_detector_model/checkpoint-364' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Device set to use cuda:0


--- TESTING ON TRAINED DISTRIBUTION (GPT-Neo) ---
Text Snippet: successful in making a significant contribution to the present day energy needs of the world. Wind e...
Result: [{'label': 'LABEL_1', 'score': 0.9998268485069275}]


In [40]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

best_checkpoint = "/content/ai_detector_model/checkpoint-364"

print(f"Loading best model from: {best_checkpoint}")
model = AutoModelForSequenceClassification.from_pretrained(best_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(best_checkpoint)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

tests = [
    # 1. Easy AI (ChatGPT style)
    "In conclusion, machine learning is a pivotal technology that enables computers to learn from data.",

    # 2. Easy Human (Casual/Messy)
    "i honestly dont know why this is broken, maybe i missed a semicolon somewhere lol.",

    # 3. Tricky Human (Formal/Academic - The true test)
    # If this is flagged as AI, your model is biased against smart humans.
    "The industrial revolution marked a major turning point in history; almost every aspect of daily life was influenced in some way.",

    # 4. Tricky AI (Creative/Story)
    # AI trying to sound human.
    "The old man sat on the porch, watching the sun dip below the horizon. 'It's been a long time,' he whispered."
]

print("\n--- FINAL EVALUATION ---")
for i, text in enumerate(tests):
    result = classifier(text)
    label = result[0]['label']
    score = result[0]['score']
    # Map LABEL_1 to AI, LABEL_0 to Human
    verdict = "AI   " if label == "LABEL_1" else "Human"
    print(f"Test {i+1}: {verdict} (Confidence: {score:.4f})")

Loading best model from: /content/ai_detector_model/checkpoint-364


The tokenizer you are loading from '/content/ai_detector_model/checkpoint-364' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Device set to use cuda:0



--- FINAL EVALUATION ---
Test 1: Human (Confidence: 0.5418)
Test 2: Human (Confidence: 0.6412)
Test 3: Human (Confidence: 0.7463)
Test 4: Human (Confidence: 0.6618)


Uploading Model to Huggingface

In [ ]:
!huggingface-cli login

In [37]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 1. SETUP PATHS
# Use the exact folder name of your best model (e.g., checkpoint-364)
local_model_path = "/content/ai_detector_model/checkpoint-364"

# Replace 'your-username' with your actual Hugging Face username
# Replace 'project-name' with what you want to call it
repo_name = "vraj33/ai-text-detector-deberta"

print(f"Loading model from {local_model_path}...")

# 2. LOAD
# We load it first to ensure we aren't uploading broken files
model = AutoModelForSequenceClassification.from_pretrained(local_model_path)
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# 3. PUSH TO HUB
print(f"Uploading to Hugging Face: {repo_name}...")
# This creates the repo if it doesn't exist
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"✅ Success! Your model is live at: https://huggingface.co/{repo_name}")

Loading model from /content/ai_detector_model/checkpoint-364...


The tokenizer you are loading from '/content/ai_detector_model/checkpoint-364' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Uploading to Hugging Face: vraj33/ai-text-detector-deberta...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  /tmp/tmpc395vi8_/spm.model  : 100%|##########| 2.46MB / 2.46MB            

✅ Success! Your model is live at: https://huggingface.co/vraj33/ai-text-detector-deberta
